```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
import matplotlib.pyplot as plt

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/motion_compensation.ipynb>`, or a {download}`python script <converted/motion_compensation.py>` with code cells.
```

# Tutorial 4: Motion Compensation

This tutorial introduces the motion compensation capabilities of ``optimap``. Motion in optical mapping studies causes severe measurement artifacts, often referred to as 'motion artifacts' [{footcite:t}`Rohde2005,Christoph2018a, Lebert2022, Kappadan2023, Christoph2023`]. To avoid these artifacts, the vast majority of optical mapping studies are conducted with pharmacological agents, such as Blebbistatin, which suppress the contractile motion of heart tissue by uncoupling the excitation-contraction coupling mechanism [{footcite:t}`Swift2021`]. `optimap` includes numerical methods with which motion artifacts can be substantially reduced. Instead of suppressing motion pharmacologically, we can use ``optimap`` to track, stabilize and inhibit motion and motion artifacts numerically. In many cases, uncoupling agents are no longer needed and optical mapping studies can be performed with freely contracting tissues. 

This tutorial explains how to process cardiac optical mapping recordings with motion and discusses best practices to reduce motion artifacts using numerical motion tracking. [Tutorial 7](ratiometry.ipynb) discusses an additional optical technique, ratiometric imaging, which can be applied in combination with numerical motion tracking to further reduce motion artifacts, see also {footcite:t}`Kappadan2020`.

We will discuss several examples which illustrate the effectiveness of numerical motion compensation as well as its limitations. 

Motion can be compensated with just 1 line of code in ``optimap``:

In [ ]:
video_compensated = om.motion_compensate(video)

## Types of Video Data and Rhythms

Numerical motion tracking and artifact compensation can be performed with whole organ, cell culture, and single cells preparations. The effectiveness of numerical motion compensation depends on various factors including properties of the video data and the type of rhythm:

* Strong motion and deformation poses the most difficult data, because the tissue can move out of the field of view or deform so strongly that tracking fails. With strong motion (e.g. sinus rhythm) it is also required to use ratiometric imaging in addition to numerical motion compensation, see [Tutorial 7](ratiometry.ipynb) for more details.
* Arrhythmias, in particular tachyarrhythmias such as ventricular fibrillation (VF), are easier to process because motion is moderate or small, see {footcite:t}`Christoph2018,Christoph2018a`. Ratiometric imaging, as described in [Tutorial 7](ratiometry.ipynb), is not necessarily required during arrhythmias depending on the motion and desired analysis, see also Fig. 12 in {footcite:t}`Kappadan2020`.
* Measuring action potential durations (APD) (see [Tutorial 8](apd.ipynb)) with motion is much more challenging than measuring activation times/maps (see [Tutorial 5](activation.ipynb)) or conduction velocities (see [Tutorial 6](cv.ipynb)). Action potential duration (APD) measurements with contracting tissues require ratiometric imaging, see [Tutorial 7](ratiometry.ipynb) and Figs. 1-7 in {footcite:t}`Kappadan2020`.
* Noisy video data can impede the tracking, see {footcite:t}`Lebert2022`.
* A shallow depth of focus and blurring can impede the tracking.
* Tracking tissue close to the video image boundaries is prone to fail. Keep the tissue centered and leave enough space around it during imaging.
* The tissue needs to be illuminated as evenly as possible, see [Tutorial 7](ratiometry.ipynb) for details.

### Small Motion

Relatively little contractile motion occurs during tachyarrhythmias, such as ventricular fibrilaltion (VF) in isolated hearts, or in cell cultures. Such small or finite motion promises the highest efficacies when trying to compensate motion artifacts using numerical motion tracking.
[Tutorial 1](basics.ipynb) already demonstrated numerical motion compensation with ventricular fibrillation (VF). The recording was obtained without Blebbistatin. The dominant frequency of the vortex waves is very high (> 10Hz) and, accordingly, the contractile motion is very small. Numerical motion compensation works very well under these circumstances provided the video quality is sufficiently high and other properties are right. In this VF example, motion artifacts are very strong without numerical motion compensation despite the minimal motion. After numerical motion compensation, it is possible to normalize the data, visualize waves, calculate phase maps, see [Tutorial 9](phase.ipynb), and calculate dominant frequencies. The following lines of code load the VF example file from our website [cardiacvision.ucsf.edu](https://cardiacvision.ucsf.edu), perform the motion compensation and display the result:

In [ ]:
import optimap as om

filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um.npy')
video_VF = om.load_video(filepath)
video_VF = video_VF[:500] # overload only the first 500 frames
video_VF_compensated = om.motion_compensate(video_VF, contrast_kernel=5, presmooth_spatial=1, presmooth_temporal=1)

In [ ]:
om.video.playn([video_VF, video_VF_compensated], titles=["original video", "compensated"], figsize=(6, 3.5));

In [ ]:
render_ani_func(lambda: om.video.playn([video_VF, video_VF_compensated], titles=["original video", "compensated"], figsize=(6, 3.5)))

The pixel-wise normalized videos show action potential vortex waves in the motion compensated videos and motion artifacts in the uncompensated videos:

In [ ]:
video_VF_compensated_norm = om.video.normalize_pixelwise_slidingwindow(video_VF_compensated, window_size=60)
video_VF_norm = om.video.normalize_pixelwise_slidingwindow(video_VF, window_size=60)

In [ ]:
om.video.playn([video_VF_compensated_norm, video_VF_norm], titles=["compensated", "uncompensated"], figsize=(6, 3.5));

In [ ]:
render_ani_func(lambda: om.video.playn([video_VF_compensated_norm, video_VF_norm], titles=["compensated", "uncompensated"], figsize=(6, 3.5)))

The VF example highlights that motion artifacts can occur with very minimal motion and that numerical motion tracking can compensate these motion artifacts very effectively, see also {footcite:t}`Christoph2018a, Lebert2022`.

### Strong Motion

Videos of sinus rhythm or pacing often include much stronger motion and deformation. Stronger motion and deformation is more challenging to track and process for several reasons. Nevertheless, it is possible to track data with strong motion, given that certain prerequisites are met. The following lines of code load an example video file of sinus rhythm from our website [cardiacvision.ucsf.edu](https://cardiacvision.ucsf.edu) and perform motion compensation:

In [ ]:
import optimap as om

filename = om.utils.retrieve_example_data('Example_01_Sinus_Rabbit_Basler.npy')
video_sinus = om.load_video(filename)
video_sinus = om.video.rotate_left(video_sinus)
video_sinus_compensated_ref0 = om.motion_compensate(video_sinus, contrast_kernel=5, ref_frame=0)
video_sinus_compensated_ref40 = om.motion_compensate(video_sinus, contrast_kernel=5, ref_frame=40)

We actually performed the motion tracking and compensation twice: once with reference frame 0 (the first frame in the video) and once with reference frame 40 (the video contains only 90 frames). The reference frame can be specified in {func}`motion_compensate`: tracking will be performed with respect to this frame, which means that the tissue's displacements will be calculated with respect to the tissue's mechanical configuration in this frame, and all other video frames will be 'warped' onto (will look similar to) this frame using the displacement vector field data. You can see that the choice of the reference frame can influence the motion compensation:

In [ ]:
om.video.playn([video_sinus, video_sinus_compensated_ref40, video_sinus_compensated_ref0], titles=["original video", "compensated ref 40", "compensated ref 0"], figsize=(8, 3.5));

In [ ]:
render_ani_func(lambda: om.video.playn([video_sinus, video_sinus_compensated_ref40, video_sinus_compensated_ref0], titles=["original video", "compensated ref 40", "compensated ref 0"], interval=20, figsize=(8, 3.5)))

With reference frame 40 the compensated video is completely motion-stabilized and we could proceed with post-processing. However, with reference frame 0 the compensated video exhibits warping artifacts. This is because the motion is much larger from frame 0 all the way to the end of the video than from one of the frames halfway through the video (particularly in this example, overall this is not always an issue). With reference frame 0 the tracking algorithm was not able to establish a spatial correlation with all frames. With sinus rhythm or pacing it is usually good practice to pick a reference frame during diastly shortly before the depolarization phase or application of a pacing stimuli, as this frame shows the tissue in a relaxed, uncontracted state (relevant when computing tissue strain or contraction). However, depending on the magnitude of translational and/or rotational motion and deformation it might be required to pay attention to whether the camera can see the tissue throughout the entire sequence of frames. Selecting a different reference frame can circumvent issues with the tracking. By contrast, during VF it is not necessarily required to pick a reference frame, any frame will work as long as the tissue does not move much. A related issue is that with sinus rhythm the heart might rotate during contraction such that parts of the tracked tissue do not face the camera any longer. Subsequently, only frames in which the tissue is fully visible can be tracked, not the entire video sequence. The latter problem can only be addressed using multi-camera optical mapping systems as in {footcite:t}`Chowdhary2023`.

The original and the compensated pixel-wise normalized videos look as follows:

In [ ]:
video_sinus_norm = om.video.normalize_pixelwise(video_sinus[:25])
video_sinus_compensated_ref40_norm = om.video.normalize_pixelwise(video_sinus_compensated_ref40[:25])

In [ ]:
om.video.playn([video_sinus_norm, video_sinus_compensated_ref40_norm], titles=["original video", "compensated ref 40"], figsize=(6.5, 3.5), interval=100);

In [ ]:
render_ani_func(lambda: om.video.playn([video_sinus_norm, video_sinus_compensated_ref40_norm], titles=["original video", "compensated ref 40"], figsize=(6.5, 3.5), interval=100))

The difference between the uncompensated and compensated videos before and after motion tracking is striking. However, motion artifacts are not entirely removed with numerical motion tracking with large motion. Note that we selected only the first 25 frames during the normalization. Selecting such a short window, which shows only the depolarization wave front propagating across the tissue, helps suppress additional motion artifacts. It is then possible to compute activation maps from the short compensated video, see [Tutorial 5](activation.ipynb). However, if you want to measure action potential durations (APDs) in longer videos you need to combine the numerical motion compensation with ratiometric imaging, see [Tutorial 7](ratiometry.ipynb) and [Tutorial 8](apd.ipynb).

### Customizing Motion Tracking

The success of motion tracking depends on various other factors such as noise, image contrast, blurring, illumination, etc., see {footcite:t}`Lebert2022`. Internally, the {func}`motion_compensate` function executes several subroutines, including {func}`estimate_displacements`, {func}`contrast_enhancement`, {func}`smooth_spatiotemporal` and {func}`warp_video`, which influence the motion tracking. In particular, contrast-enhancement and spatio-temporal smoothing are two pre-processing steps, which we strongly recommend, to increase the robustness of the tracking. To adapt the processing to particular video data, it is either possible to provide parameters to {func}`motion_compensate` (e.g. `contrast_kernel=5`) or to execute the subroutines individually one after another. Instead of:

In [ ]:
video_sinus_compensated = om.motion_compensate(video_sinus, contrast_kernel=5, ref_frame=40)

one can also use:

In [ ]:
video_sinus_smoothed = om.video.smooth_spatiotemporal(video_sinus)
video_sinus_smoothed_contrast = om.motion.contrast_enhancement(video_sinus_smoothed, 5)
displacement_vectors_sinus = om.motion.estimate_displacements(video_sinus_smoothed_contrast, 40)
video_sinus_compensated = om.motion.warp_video(video_sinus, displacement_vectors_sinus)

```{warning}
This tutorial is currently under development. We will add more information soon.
```
In particular, contrast-enhancement and spatio-temporal smoothing are very important pre-processing steps for tracking to succeed:

* Spatio-temporal smoothing of the input video (here `video_sinus`) reduces noise in the video, which subsequently also reduces noise in the tracking (jitter of the displacement vectors). The pre-smoothing can be specified in {func}`motion_compensate` (setting `presmooth_spatial=1,presmooth_temporal=1` as input parameters) or be performed at first before the other subroutines if the subroutines are called sequentially, see below.

* Contrast-enhancement increases the robustness of the tracking and prevents tracking artifacts. {func}`motion_compensate` performs contrast-enhancement internally by default and tracks motion in a contrast-enhanced was introduced in {footcite:t}`Christoph2018a`.important pre-processing routine and parameter that needs to be specified is contrast-enhancement and the size of the kernel used for the contrast-enhancement, respectively, see {footcite:t}`Christoph2018a, Lebert2022`. and varied depending on the size of the video image and the size of visible structures in the image (heart, vessels, cell, etc.) is the contrast-enhancement parameter:

In [ ]:
video_sinus_contrast = om.motion.contrast_enhancement(video_sinus, 5)

sinus_displacement_vectors = om.motion.estimate_displacements(video_sinus, 40)
sinus_displacement_vectors_contrast = om.motion.estimate_displacements(video_sinus_contrast, 40)

video_sinus_compensated_nocontrast = om.motion.warp_video(video_sinus, sinus_displacement_vectors)
video_sinus_compensated_contrast = om.motion.warp_video(video_sinus, sinus_displacement_vectors_contrast)

In [ ]:
om.video.playn([video_sinus, video_sinus_compensated_contrast, video_sinus_compensated_nocontrast],
               titles=["original video", "with contrast-enhancement", "w/o contrast-enhancement"], figsize=(8, 3.5));

In [ ]:
render_ani_func(lambda: om.video.playn([video_sinus, video_sinus_compensated_contrast, video_sinus_compensated_nocontrast],
               titles=["original video", "with contrast-enhancement", "w/o contrast-enhancement"], figsize=(8, 3.5)));

The kernel size is crucial and needs to be adapted to the image properties:

In [ ]:
contrast3 = om.motion.contrast_enhancement(video_sinus, 3)
contrast5 = om.motion.contrast_enhancement(video_sinus, 5)
contrast9 = om.motion.contrast_enhancement(video_sinus, 9)
om.video.playn([contrast3, contrast5, contrast9],
               titles=["contrast kernel 3", "contrast kernel 5", "contrast kernel 9"],
               skip_frame=3,
               figsize=(8, 3.5));

In [ ]:
def f():
    contrast3 = om.motion.contrast_enhancement(video_sinus, 3)
    contrast5 = om.motion.contrast_enhancement(video_sinus, 5)
    contrast9 = om.motion.contrast_enhancement(video_sinus, 9)
    return om.video.playn([contrast3, contrast5, contrast9], titles=["contrast kernel 3", "contrast kernel 5", "contrast kernel 9"], skip_frame=1, figsize=(8, 3.5))
render_ani_func(f)

By default, we chose a contrast kernel size of 5.

### Retrieving Displacement Data for Analysis of Tissue Mechanics

The code below produces 2 videos, one with and one without contrast-enhancement:


The video shows an action potential wave propagating across the ventricular surface of beating rabbit heart. The recording was peformed with voltage-sensitive fluorescent dye (Di-4-ANEPPS) and a high-speed camera (Basler acA720-520um) at 500fps. Experimenters: Jan Lebert, Namita Ravi & Jan Christoph (University of California, San Francisco, USA), 2022.

Alternatively, you can load your own video file by replacing `filename` with the name of your file (e.g. `filename = 'your_video_file.rsh'`), if it is located in the same folder as the script. If the video file is located somewhere else on your computer you can provide the path and filename (e.g. `'filename = /Users/userx/Desktop/your_video_file.rsh'`). Note, that we used {func}`optimap.video.rotate_left` to rotate the video to the left.

```{footbibliography}
```